## config

In [1]:
import os
import glob
import re

In [2]:
path = "/home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope"
path_mhcI = path + "/02.protein_antigen_prediction/parsed_res_MHCI"
path_mhcII = path + "/02.protein_antigen_prediction/parsed_res_MHCII"
homo_bed = path + "/03.homologous/homo_peptide.bed"
trans_memb = path + "/04.TransMembrane.DeepTMHMM/TransMembrane.bed"
enzymedigest = path + "/06.EnzymeDigest/protein.merge.enzymedigest"
outpath = path + "/05.Deliverables/EpitopePresent"
outpath1 = path + "/05.Deliverables/EpitopePresent/MHCI"
outpath2 = path + "/05.Deliverables/EpitopePresent/MHCII"
try:
    os.mkdir(outpath)
except:
    pass
try:
    os.mkdir(outpath1)
except:
    pass
try:
    os.mkdir(outpath2)
except:
    pass

In [3]:
ls /home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/05.Deliverables/

01.Protein.merge.aa.fasta       03.Homo_peptide.bed   EpitopePresent.tar.gz
02.Immunogenicity.netMHCI.bed   04.TransMembrane.bed
02.Immunogenicity.netMHCII.bed  EpitopePresent/


In [4]:
import difflib
def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return(size,s1[pos_a:pos_a+size])
s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

(39, 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC')


In [5]:
## Parser the TransMemb.bed
TM_parsDict={}
flagDict={"o":"outside","T":"TMhelix","i":"inside","s":"signal"}
'''
    parser the tranmembe.bed file
    TM_flag=outside,TMhelix,inside
    outside:o
    TMhelix:t
    inside:i
'''
with open(trans_memb,"r") as tmf:
    for line in tmf.readlines():
        if line.startswith("sp|"):
            seqid = line.split("\t")[0].split("|")[1]
            if seqid not in TM_parsDict.keys():
                TM_parsDict[seqid] = ""
            pos1 = int(line.split("\t")[1])
            pos2 = int(line.split("\t")[2])
            TM_flag = line.split("\t")[3][0]
            TM_parsDict[seqid] += TM_flag*(1+pos2-pos1)

In [6]:
#TM_parsDict

In [7]:
## Parser the Homologous_peptide.bed
Homo_parsDict={}
with open(homo_bed,"r") as hmf:
    for line in hmf.readlines():
        if line.startswith("sp|"):
            #print(line)
            seqid = line.split("\t")[0].split("|")[1]
            if seqid not in Homo_parsDict.keys():
                length = len(TM_parsDict[seqid])
                Homo_parsDict[seqid] = [0]*length
            pos1 = int(line.split("\t")[1])-1
            pos2 = int(line.split("\t")[2])-1
            homoid = line.split("\t")[3]
            for pos in range(pos1,pos2):
                #print(pos)
                if Homo_parsDict[seqid][pos] == 0:
                    Homo_parsDict[seqid][pos] = [homoid]
                else:
                    Homo_parsDict[seqid][pos].append(homoid)

In [8]:
ls /home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/05.Deliverables

01.Protein.merge.aa.fasta       03.Homo_peptide.bed   EpitopePresent.tar.gz
02.Immunogenicity.netMHCI.bed   04.TransMembrane.bed
02.Immunogenicity.netMHCII.bed  EpitopePresent/


In [9]:
#print(Homo_parsDict)

In [10]:
a="MMFPLARNALSSLKIQSILQSMARHSHVKHSPDFHDKYGNAVLASGTAFCVATWVFTATQIGIEWNLSPVGRVTPKEWKHQ"
b=["HVKHSPDFHDK","FPLARNAL","SPVGRVTPKEW"]
for i in b:
    print(a.find(i))
    

26
2
67


In [11]:
## Parser the enzymedigest file
EnzDigeDict={}
with open(enzymedigest,"r") as enzymef:
    for line in enzymef.readlines():
        refseq = re.split(',"\[',line)[0].split(",")[-1]
        refid = re.split(',"\[',line)[0].split("|")[1]
        seqlist = re.split(',"\[',line)[1].strip().replace(']"','').replace("'","").split(",")
        EnzDigeDict[refid]=seqlist
print(EnzDigeDict)

{'P30939': ['EFKHEKSW', ' IIVTRKLHHP', ' NSLINPLIYTI', ' NRMPSKILV', ' CSILHLSAIA', ' KIYRAAKTLYHK', ' SEFKHEKSW', ' IVYIVRESW', ' NSLINPLIY', ' KEEVNGQVL', ' STKSVSTSY', ' LVMPFSIVY', ' FYIPLALIL', ' LINPLIYTI'], 'Q00534': ['ETFEHPNVV', ' ETFEHPNVVRL', ' QMALTSVVV', ' FGLARIYSF', ' RADQQYECV', ' KLADFGLARIYSFQMALTSVVVTLW', ' LTSVVVTLW', ' DFGLARIY', ' SRTDRETKLTLVF', ' RTDRETKL', ' MPLSTIREV', ' RIYSFQMAL', ' HVDQDLTTY', ' MALTSVVV', ' KSAQPIEKF', ' QQYECVAEI', ' HRDLKPQNI', ' ALTSVVVTL', ' KLADFGLAR', ' ALPRQAFHSKSAQPIEKF', ' KPQNILVTS', ' HRVVHRDL', ' LSTIREVAV', ' RADQQYECVAEIGEGAYGKVF', ' SRTDRETKL', ' KLADFGLARI', ' STIREVAVL', ' ETFEHPNVVR', ' RETKLTLVF', ' HSKSAQPIEK', ' LGKDLLLK', ' AEIGEGAYGKVF', ' FEHPNVVRL', ' GMPLSTIREV', ' GEGAYGKVF', ' HRVVHRDLKPQNILVTS', ' DELGKDLLL', ' AYSALSHPYF', ' LARIYSFQM', ' DELGKDLLLK', ' FGLARIYSFQMALTSVVVTLW', ' ALPRQAFHSK', ' GMPLSTIREVAV', ' GMPLSTIREVA', ' FEHPNVVR', ' MPLSTIREVA', ' AYSALSHPY'], 'Q5HYN5': ['TDKTEKVAV', ' SQREINADIKR', ' KT

In [12]:
ls /home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/06.EnzymeDisgest/protein.merge.enzymedigest

ls: cannot access '/home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/06.EnzymeDisgest/protein.merge.enzymedigest': No such file or directory


In [13]:
## Parser the netMHC.csv
mergeoutfile =  open(outpath+"/MHCI_Epitopeonly_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcI + "/*.netMHC.csv"):
    filenum +=1
    filename= os.path.basename(file)
    outfile = open(outpath1+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            """
            itemlist is the line content of netMHC result
            """
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                seqid = itemlist[10].split("_")[1]
                midpos = int((pos1+pos2)/2)
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = str(size0)+"/"+str(len(peptide))
                Cut_seq = overseq0
                
                ## get homolgous info
                checklist = [homo_f==0 for homo_f in Homo_parsDict[seqid][pos1:pos2]]
                if  sum(checklist)/(pos2-pos1) >= 0.05:
                    HomoEx = "N"
                    Homo_flag = ""
                else:
                    HomoEx = "Y"
                    flagtemp_list = []
                    for flag in Homo_parsDict[seqid][pos1:pos2]:
                        if flag !=0:
                            flagtemp_list += flag
                    Homo_flag = ";".join(list(dict.fromkeys(flagtemp_list)))
                    
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,Cut_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()


In [14]:
## Parser the netMHC.csv
mergeoutfile2 =  open(outpath+"/MHCII_Epitope_only_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcII + "/*.netMHC.csv"):
    filenum += 1
    filename= os.path.basename(file)
    outfile = open(outpath2+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                seqid = itemlist[6].split("|")[1]
                midpos = int((pos1+pos2)/2)
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = str(size0)+"/"+str(len(peptide))
                Cut_seq = overseq0
                
                ## get homolgous info
                checklist = [homo_f==0 for homo_f in Homo_parsDict[seqid][pos1:pos2]]
                if  sum(checklist)/(pos2-pos1) >= 0.05:
                    HomoEx = "N"
                    Homo_flag = ""
                else:
                    HomoEx = "Y"
                    flagtemp_list = []
                    for flag in Homo_parsDict[seqid][pos1:pos2]:
                        if flag !=0:
                            flagtemp_list += flag
                    Homo_flag = ";".join(list(dict.fromkeys(flagtemp_list)))
                        
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

In [15]:
mylist = ["a", "b", "a", "c", "c"]
mylist = list(dict.fromkeys(mylist))
print(mylist)

['a', 'b', 'c']


In [16]:
";".join(list(dict.fromkeys(mylist)))

'a;b;c'

In [17]:
fasta = 'CGTAGCGGCTGAGTGCGCGGATAGCGCGTA'
target = 'CGGCTGAGT'

start = fasta.find(target)+1
end = start + len(target) -1

print(start, end)

6 14


In [18]:
import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC
